In [ ]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [2]:
import tensorflow as tf
from sklearn import cross_validation
from sklearn import metrics as mt
from sklearn import utils as ut
import csv
import numpy as np
import os.path as path
from PIL import Image
from skimage.color import rgb2gray

In [3]:
# Load data
featuresDir = './data/SampleFoodClassifier_Norm_100'

with open('./sample_food_no_food.csv') as f:
    food_no_food = [{k: v for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    

data_ids = [element['id'] for element in food_no_food]

labels = [int(element['is_food']) for element in food_no_food]
#data = [rgb2gray(np.array(Image.open(path.join(featuresDir, element)))) for element in data_ids]
data = [np.array(Image.open(path.join(featuresDir, element))) for element in data_ids]

In [4]:
# Split training data in a train set and a test set. The test set will containt 20% of the total
x_train, x_test, y_train, y_test = cross_validation.train_test_split(data, labels, test_size=.25, random_state=6)

# Parameters
#learning_rate = 0.001
learning_rate_start= .001
training_size = 30
training_split= 0.8 #split into training_split 1 and (1-training_split) 0
training_iters = 5

training_size_1=int(np.floor(training_split*training_size))
training_size_0=int(training_size-training_size_1)

print "Training size used for class 0:", training_size_0
print "Training size used for class 1:", training_size_1

# Network Parameters
w, h, channels = data[0].shape
print "Width, Height and channels:", w, h, channels
n_classes = len(set(y_train))
dropout = 0.75 # Dropout, probability to keep units

# print 'Input vector size', n_input, 'train shape', np.array(x_train).shape , 'number of classes', n_classes

# tf Graph input
x = tf.placeholder(tf.float32, [None, w, h, channels])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

Training size used for class 0: 6
Training size used for class 1: 24
Width, Height and channels: 100 100 3


In [5]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    print "PLEASE MODIFY WD1 TO", conv2.get_shape().as_list()[1], "*",conv2.get_shape().as_list()[2], "*64"

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [6]:
# Store layers weight & bias
sdev= 0.01
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.truncated_normal([5, 5, channels, 32], stddev=sdev)),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=sdev)),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.truncated_normal([25*25*64, 1024], stddev=sdev)),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.truncated_normal([1024, n_classes], stddev=sdev))
}

biases = {
    'bc1': tf.Variable(tf.truncated_normal([32], stddev=sdev)),
    'bc2': tf.Variable(tf.truncated_normal([64], stddev=sdev)),
    'bd1': tf.Variable(tf.truncated_normal([1024], stddev=sdev)),
    'out': tf.Variable(tf.truncated_normal([n_classes], stddev=sdev))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))

# optimizer without adapted learning_rate
#optimizer = tf.train.AdamOptimizerOptimizer(learning_rate=learning_rate).minimize(cost)

#optimizer with adapted learning_rate
step = tf.Variable(0, trainable=False)
rate = tf.train.exponential_decay(learning_rate_start, step, 1, 0.9999)

optimizer = tf.train.AdamOptimizer(rate).minimize(cost, global_step=step)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
y_p = tf.argmax(pred, 1)


# Gives an array of arrays, where each position represents % of belonging to respective classs. Eg: a[0.34, 0.66] --> class 0 : 34%, class 1: 66%
classes = tf.nn.softmax(pred)


def label_class(x):
    for i in range(0,len(x)):
        print i, ":", x[i]

# Initializing the variables
init = tf.initialize_all_variables()

PLEASE MODIFY WD1 TO 25 * 25 *64


In [7]:
y_train_temp = []

for element in y_train:
    temp = [0]*len(set(y_train))
    temp[element] = 1
    y_train_temp.append(temp)
    
y_train = np.reshape(y_train_temp,(len(y_train_temp), -1))

y_test_temp = []

for element in y_test:
    temp = [0]*len(set(y_test))
    temp[element] = 1
    y_test_temp.append(temp)
    
y_test = np.reshape(y_test_temp,(len(y_test_temp), -1))

# Proportional sampling from both classes, get features for 0 and 1 each
y_help=np.array([el[1] for el in y_train])
y_index_0 = np.where(y_help==0)[0]
y_index_1 = np.where(y_help==1)[0]

x_0 = [x_train[index] for index in y_index_0]
x_1 = [x_train[index] for index in y_index_1]

#y batch looks always the same for if using proportional sampling
iy= np.vstack(([[1,0]]*training_size_0,[[0,1]]*training_size_1))

In [10]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Keep training until reach max iterations
    for epoch in range(training_iters):
        # Fit training using batch data
        x_batch_0 = ut.shuffle(x_0, n_samples=training_size_0, random_state=epoch)
        x_batch_1 = ut.shuffle(x_1, n_samples=training_size_1, random_state=epoch)
        ix = x_batch_0+x_batch_1
        
        #ix = ut.shuffle(x_train, n_samples=training_size, random_state=epoch)
        #iy = ut.shuffle(y_train, n_samples=training_size, random_state=epoch)
        
        sess.run(optimizer, feed_dict={x: ix, y: iy, keep_prob: 1.})
        # Compute average loss
        loss, acc = sess.run([cost, accuracy], feed_dict={x: ix, y: iy, keep_prob: 1.})
        # Display logs per epoch step
        print "Iter " + str(epoch) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
    print "Optimization Finished!"

    runs = 0
    acc = 0.
    y_pred = []
    class_pred = []
    for i in range(0, len(y_test), 10):
        if i+10 < len(y_test):
            val_accuracy, y_pred_i, cls = sess.run([accuracy, y_p, classes], feed_dict={x: x_test[i:i+10], y: y_test[i:i+10], keep_prob: 1.})
            acc += val_accuracy
            y_pred.extend(y_pred_i)
            class_pred.extend(cls)
        else:
            val_accuracy, y_pred_i, cls = sess.run([accuracy, y_p, classes], feed_dict={x: x_test[i:], y: y_test[i:], keep_prob: 1.})
            acc += val_accuracy
            y_pred.extend(y_pred_i)
            class_pred.extend(cls)
        runs += 1
        print "Partial testing accuracy:", acc/runs
    
    #metrics
    print "Validation accuracy:", acc/runs
    y_true = np.argmax(y_test,1)
    print "Precision for each class:"
    label_class(mt.precision_score(y_true, y_pred, average=None))
    print "Recall for each class:"
    label_class(mt.recall_score(y_true, y_pred, average=None))
    print "F1_score for each class:"
    label_class(mt.f1_score(y_true, y_pred, average=None))
    print "confusion_matrix"
    print mt.confusion_matrix(y_true, y_pred)
    fpr, tpr, tresholds = mt.roc_curve(y_true, y_pred)

Iter 0, Minibatch Loss= 50.314503, Training Accuracy= 0.80000
Iter 1, Minibatch Loss= 1.062663, Training Accuracy= 0.83333
Iter 2, Minibatch Loss= 12.905790, Training Accuracy= 0.20000
Iter 3, Minibatch Loss= 2.619724, Training Accuracy= 0.20000
Iter 4, Minibatch Loss= 0.628211, Training Accuracy= 0.80000
Optimization Finished!
Partial testing accuracy: 1.0
Partial testing accuracy: 0.949999988079
Partial testing accuracy: 0.833333333333
Partial testing accuracy: 0.82500000298
Partial testing accuracy: 0.860000002384
Partial testing accuracy: 0.833333333333
Partial testing accuracy: 0.828571430274
Partial testing accuracy: 0.80000000447
Partial testing accuracy: 0.800000005298
Partial testing accuracy: 0.810000002384
Partial testing accuracy: 0.82727272944
Partial testing accuracy: 0.81666666766
Partial testing accuracy: 0.807692307692
Partial testing accuracy: 0.821428571429
Partial testing accuracy: 0.820000000795
Partial testing accuracy: 0.81875000149
Partial testing accuracy: 0.81

In [11]:
for i in range(len(y_test)):
    print "For", i, "as", y_test[i][1], "predicted to be 0 at:", class_pred[i][0]*100, "%, predicted to be 1 at: ", class_pred[i][1]*100

For 0 as 1 predicted to be 0 at: 46.893247962 %, predicted to be 1 at:  53.1067490578
For 1 as 1 predicted to be 0 at: 45.5796957016 %, predicted to be 1 at:  54.4203102589
For 2 as 1 predicted to be 0 at: 45.3116029501 %, predicted to be 1 at:  54.6884000301
For 3 as 1 predicted to be 0 at: 41.9941604137 %, predicted to be 1 at:  58.0058395863
For 4 as 1 predicted to be 0 at: 44.8121964931 %, predicted to be 1 at:  55.1877975464
For 5 as 1 predicted to be 0 at: 46.2605029345 %, predicted to be 1 at:  53.7394940853
For 6 as 1 predicted to be 0 at: 47.2345292568 %, predicted to be 1 at:  52.7654707432
For 7 as 1 predicted to be 0 at: 42.0678794384 %, predicted to be 1 at:  57.9321265221
For 8 as 1 predicted to be 0 at: 44.629278779 %, predicted to be 1 at:  55.3707242012
For 9 as 1 predicted to be 0 at: 44.6656912565 %, predicted to be 1 at:  55.3343057632
For 10 as 1 predicted to be 0 at: 44.4089502096 %, predicted to be 1 at:  55.5910527706
For 11 as 1 predicted to be 0 at: 44.8738396